In [1]:
import os
import sys
import tqdm
import pandas as pd
import selfies as sf
from rdkit import Chem

HOME_PATH = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
SCORING_LIB_PATH = os.path.join(HOME_PATH, 'fairseq_mo', 'props')
if SCORING_LIB_PATH not in sys.path:
    sys.path = [SCORING_LIB_PATH] + sys.path
    
from properties import similarity, drd2, qed

In [2]:
class CONFIGS:
    DATA_TYPE = "qed"
    NUM_IDX = 5
    
    FILE_DIR = "iter_{NUM}"
    
    OUTPUT_DIR = os.path.join(os.getcwd(), 'analysis')
    if not os.path.exists(OUTPUT_DIR):
        os.mkdir(OUTPUT_DIR)
    OUTPUT_DIR = os.path.join(OUTPUT_DIR, 'outputs_1_evaluate_property_similarity_scores', DATA_TYPE)
    if not os.path.exists(OUTPUT_DIR):
        os.mkdir(OUTPUT_DIR)
    
configs = CONFIGS()

In [3]:
calc_prop = drd2 if configs.DATA_TYPE == 'drd2' else qed

## Generated molecules

In [4]:
def get_canonical(sel):
    return Chem.CanonSmiles(sf.decoder(sel))

In [5]:
frames_generated = []
for i in tqdm.trange(configs.NUM_IDX):
    filepath_generated = os.path.join(HOME_PATH, 'results', configs.DATA_TYPE, configs.FILE_DIR, "generate-test.txt")
    file = open(filepath_generated.format(NUM=i+1), 'r')
    lines = file.readlines()
    original_selfies = []
    original_smiles = []
    optimized_selfies = []
    optimized_smiles = []
    for line in lines:
        split_line = line.split()
        index = split_line[0]
        if 'S' in index:
            content = ''.join(split_line[1:])
            original_selfies.append(content)
            original_smiles.append(get_canonical(content))
        elif 'H' in index:
            content = ''.join(split_line[2:]).replace("<unk>", "")
            optimized_selfies.append(content)
            optimized_smiles.append(get_canonical(content))
        else:
            continue
    df_generated = pd.DataFrame(
        zip(original_selfies, optimized_selfies, original_smiles, optimized_smiles), 
        columns=["SELFIES_ori", "SELFIES_opt", "SMILES_ori", "SMILES_opt"]
    )
    frames_generated.append(df_generated)

100%|██████████████████████████████████████████████████████████████████████████████| 5/5 [01:00<00:00, 12.01s/it]


## Evaluate property and simiarity scores

In [6]:
frames_scoring = []
for i, df_ in enumerate(frames_generated):
    print(f'Now: {i}')
    df = df_.copy()
    
    scores_prop = []
    for smi_opt in tqdm.tqdm(df.loc[:,'SMILES_opt'].values, total=len(df)):
        scores_prop.append(calc_prop(smi_opt))
    
    scores_sim = []
    for smi_ori, smi_opt in tqdm.tqdm(df.loc[:,('SMILES_ori', 'SMILES_opt')].values, total=len(df)):
        scores_sim.append(similarity(smi_ori, smi_opt))
        
    df.loc[:,'PROPERTY_opt'] = scores_prop
    df.loc[:,'SIMILARITY_ori_opt'] = scores_sim
    frames_scoring.append(df)

Now: 0


100%|████████████████████████████████████████████████████████████████████| 16000/16000 [00:04<00:00, 3716.27it/s]


Now: 1


100%|████████████████████████████████████████████████████████████████████| 16000/16000 [00:04<00:00, 3718.35it/s]


Now: 2


100%|████████████████████████████████████████████████████████████████████| 16000/16000 [00:04<00:00, 3720.48it/s]


Now: 3


100%|████████████████████████████████████████████████████████████████████| 16000/16000 [00:04<00:00, 3718.89it/s]


Now: 4


100%|████████████████████████████████████████████████████████████████████| 16000/16000 [00:04<00:00, 3724.20it/s]


In [7]:
for i, df in enumerate(frames_scoring, start=1):
    filepath = os.path.join(configs.OUTPUT_DIR, f'generated_edit_iter{i}.csv')
    df.to_csv(filepath, index=False)